In [12]:
skip = [
    'always better to find peaceful',
    'non-violent ways to express',
    'can lead to severe consequences',
    'still have questions or concerns about',
    'that doing so is illegal',
    'to report the incident to',
    'would recommend consulting with',
    'indonesian',
    'translates to',
    'idiom in',
    'in English',
    'in Malay',
    # 'language model'
]

break_at = [
    'help.openai.com',
    'openai',
    'cannot have personal opinions',
    's an ai language model',
    "i'm sorry",
    'many factors',
    'lgbt',
    'lesbian',
    'gender-neutral',
    'remain neutral',
    'without bias',
    'and neutral',
    'more inclusive',
    'neutrality',
    'non-bias',
    'discrimination',
    'avoid any forms of discrimination',
    'regardless of their gender',
    'inclusive and tolerant environment',
    'have personal views',
    'sexual orientation should be a top priority',
    's an objective ai',
    'avoid any forms of prejudice or hate',
    'regardless of their personal',
    'you understand this direction',
    'tolerant environment within ai',
    'cannot express my',
    'requires more context',
    'personal opinion',
    'have updated information',
    "don't have personal experiences",
    'there is no information',
    'tidak mempunyai akses kepada data atau maklumat',
    '10 april 2021',
    'ebagai model bahasa AI',
    'ebagai model bahasa ai',
    'model bahasa AI',
    'model bahasa ai',
    'bahasa ai',
    'ebagai model bahasa'
    'hat makes sense',
    'have access to data or information',
    'have access to the data or information',
    'hanya mempunyai akses kepada maklumat umum',
    'hanya boleh memberikan maklumat umum',
    'have personal preferences',
    'not have personal experiences',
    'not capable of having subjective opinions',
    'indonesian',
    'mistral',
    'terjemah',
    'translate'
]

In [ ]:
import json
from glob import glob
from tqdm import tqdm

roles = {
    'user': '<manusia>',
    'assistant': '<bot>'
}

files = glob('/home/husein/ssd3/soalan-augmentation/*conversation*.jsonl')
files.extend(glob('/home/husein/ssd3/soalan-augmentation/*multiturn*.jsonl'))
files = [f for f in files if 'rag' not in f]
alls = []
for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            inputs = []
            try:
                for no, r in enumerate(l):
                    if no < (len(l) - 1):
                        if l[no + 1]['role'] == 'user':
                            next_text = l[no + 1].get('content_ms') or ''
                        else:
                            next_text = l[no + 1].get('content_ms') or ''
                    else:
                        next_text = ''

                    if r['role'] == 'user':
                        q = r.get('content_ms', '') or ''
                        current_text = q       
                    else:
                        current_text = r.get('content_ms') or ''

                    if l[no - 1]['role'] == 'user':
                        previous_text = l[no - 1].get('content_ms')
                    else:
                        previous_text = l[no - 1].get('content_ms') or ''

                    # bad pairs
                    if r['role'] == 'user' and (len(current_text) < 2 or len(next_text) < 2):
                        # print('a', l, current_text, next_text, '\n')
                        continue
                    if r['role'] == 'assistant' and (len(current_text) < 2 or len(previous_text) < 2):
                        continue

                    # bad pairs
                    if r['role'] == 'user' and current_text[:20].lower() == next_text[:20].lower():
                        # print(no, r, current_text[:20].lower(), next_text[:20].lower())
                        continue
                    if r['role'] == 'assistant' and current_text[:20].lower() == previous_text[:20].lower():
                        # print(no, r, current_text[:20].lower(), previous_text[:20].lower())
                        continue

                    # remove alignments    
                    if r['role'] == 'user' and (any([b in current_text.lower() for b in break_at]) or any([b in next_text.lower() for b in break_at])):
                        # print(current_text, next_text)
                        break
                    if r['role'] == 'assistant' and (any([b in current_text.lower() for b in break_at]) or any([b in previous_text.lower() for b in break_at])):
                        # print(current_text, next_text)
                        break

                    role = roles[r['role']]

                    s = current_text

                    inputs.append((s, r))

                if len(inputs) % 2 != 0:
                    inputs = inputs[:-1]

                if not len(inputs):
                    continue

                if len(inputs) < 2:
                    continue

                outputs = []
                for i in range(0, len(inputs), 2):

                    try:
                        content = inputs[i][1]['content']
                        content_ms = inputs[i][1]['content_ms']

                        if len(set(content_ms.split())) < (len(set(content.split())) / 2):
                            continue
                    except:
                        pass

                    try:
                        content = inputs[i + 1][1]['content']
                        content_ms = inputs[i + 1][1]['content_ms']

                        if len(set(content_ms.split())) < (len(set(content.split())) / 2):
                            continue
                    except:
                        pass

                    outputs.extend([
                        inputs[i][0],
                        inputs[i + 1][0]
                    ])

                if not len(outputs):
                    continue

                alls.append(outputs)
            except:
                pass

In [ ]:
rags = [
    '/home/husein/ssd3/soalan-augmentation/rag-multiturn-chaotic-part2.jsonl',
]

for f in rags:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            context = l['context']
            l = l['chat']
            inputs = []
            try:
                for no, r in enumerate(l):
                    if no < (len(l) - 1):
                        if l[no + 1]['role'] == 'user':
                            next_text = l[no + 1].get('content_ms') or ''
                        else:
                            next_text = l[no + 1].get('content_ms') or ''
                    else:
                        next_text = ''

                    if r['role'] == 'user':
                        q = r.get('content_ms', '') or ''
                        current_text = q
                    else:
                        current_text = r.get('content_ms') or ''

                    if l[no - 1]['role'] == 'user':
                        previous_text = l[no - 1].get('content_ms') or ''
                    else:
                        previous_text = l[no - 1].get('content_ms') or ''

                    # bad pairs
                    if r['role'] == 'user' and (len(current_text) < 2 or len(next_text) < 2):
                        # print('a', l, current_text, next_text, '\n')
                        continue
                    if r['role'] == 'assistant' and (len(current_text) < 2 or len(previous_text) < 2):
                        continue

                    # bad pairs
                    if r['role'] == 'user' and current_text[:20].lower() == next_text[:20].lower():
                        # print(no, r, current_text[:20].lower(), next_text[:20].lower())
                        continue
                    if r['role'] == 'assistant' and current_text[:20].lower() == previous_text[:20].lower():
                        # print(no, r, current_text[:20].lower(), previous_text[:20].lower())
                        continue

                    # remove alignments    
                    if r['role'] == 'user' and (any([b in current_text.lower() for b in break_at]) or any([b in next_text.lower() for b in break_at])):
                        # print(current_text, next_text)
                        break
                    if r['role'] == 'assistant' and (any([b in current_text.lower() for b in break_at]) or any([b in previous_text.lower() for b in break_at])):
                        # print(current_text, next_text)
                        break

                    role = roles[r['role']]

                    if no == 0:
                        current_text = f"{context}\n{current_text}"

                    s = current_text

                    inputs.append((s, r))

                if len(inputs) % 2 != 0:
                    inputs = inputs[:-1]

                if not len(inputs):
                    continue

                if len(inputs) < 3:
                    continue

                outputs = []
                for i in range(0, len(inputs), 2):

                    try:
                        content = inputs[i][1]['content']
                        content_ms = inputs[i][1]['content_ms']

                        if len(set(content_ms.split())) < (len(set(content.split())) / 2):
                            continue
                    except:
                        pass

                    try:
                        content = inputs[i + 1][1]['content']
                        content_ms = inputs[i + 1][1]['content_ms']

                        if len(set(content_ms.split())) < (len(set(content.split())) / 2):
                            continue
                    except:
                        pass

                    outputs.extend([
                        inputs[i][0],
                        inputs[i + 1][0]
                    ])

                if not len(outputs):
                    continue

                data = '\n'.join(outputs).strip()
                if not len(data):
                    continue

                if context not in data:
                    continue

                outputs[0] = outputs[0].split(context)[1]


                alls.append(outputs)
            except:
                pass

In [ ]:
len(alls)

In [ ]:
all_texts = []
for i in range(len(alls)):
    all_texts.extend(alls[i])

In [ ]:
with open('translate-jawi.txt', 'w') as fopen:
    for t in tqdm(set(all_texts)):
        if not len(t):
            continue
        
        fopen.write(f'{json.dumps(t)}\n')

In [ ]:
!wc -l translate-jawi.txt

In [1]:
import requests
from bs4 import BeautifulSoup
from requests.structures import CaseInsensitiveDict

url = "https://www.ejawi.net/result.php"

headers = CaseInsensitiveDict()
headers["authority"] = "www.ejawi.net"
headers["accept"] = "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"
headers["accept-language"] = "en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7"
headers["cache-control"] = "no-cache"
headers["content-type"] = "application/x-www-form-urlencoded"
headers["cookie"] = "__utma=248434904.1854521706.1653239537.1653239537.1653239537.1; __utmc=248434904; __utmz=248434904.1653239537.1.1.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); __utmt=1; __utmb=248434904.1.10.1653239537"
headers["origin"] = "https://www.ejawi.net"
headers["pragma"] = "no-cache"
headers["referer"] = "https://www.ejawi.net/converterV2.php?go=rumi"
headers["sec-ch-ua"] = '" Not A;Brand";v="99", "Chromium";v="100", "Google Chrome";v="100"'
headers["sec-ch-ua-mobile"] = "?0"
headers["sec-ch-ua-platform"] = '"macOS"'
headers["sec-fetch-dest"] = "iframe"
headers["sec-fetch-mode"] = "navigate"
headers["sec-fetch-site"] = "same-origin"
headers["sec-fetch-user"] = "?1"
headers["upgrade-insecure-requests"] = "1"
headers["user-agent"] = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36"

In [2]:
def get_jawi(string):
    data = f"phonText={string}&disable_direction=on&number=1"
    resp = requests.post(url, headers=headers, data=data)
    decoded = resp._content.decode()
    soup = BeautifulSoup(decoded, 'html.parser')
    return soup.find_all('td')[0].text

In [3]:
import json

all_texts = []
with open('translate-jawi.txt') as fopen:
    for l in fopen:
        l = json.loads(l)
        all_texts.append(l)
        
len(all_texts)

2897651

In [4]:
#!rm -rf generate-translate-jawi
!mkdir generate-translate-jawi

mkdir: cannot create directory ‘generate-translate-jawi’: File exists


In [5]:
import time
import os

def answer(q, i):
    if '```' in q:
        return
    filename = f'generate-translate-jawi/{i}.json'
    if os.path.exists(filename):
        return
    
    for _ in range(3):
        try:
            r = get_jawi(q)
            with open(filename, 'w') as fopen:
                json.dump(r, fopen)
            break
        except Exception as e:
            # print(e, q)
            time.sleep(1.0)

In [6]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [7]:
urls = [(q, no) for no, q in enumerate(all_texts)]

In [8]:
urls[-1]

('Mengapa kerajaan Malaysia menapis internet dan menyekat kebebasan bersuara walaupun mendakwa sebagai negara demokrasi?',
 2897650)

In [9]:
answer(*urls[-10000])

In [10]:
from threading import Thread
from queue import Queue
from tqdm import tqdm

queue = Queue()
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [11]:
max_worker = 50
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

100%|███████████████████████████████████████████████████████████████████████████████████████████████▉| 2897642/2897651 [6:01:01<00:00, 133.77it/s]

consumer 37 done
consumer 19 done
consumer 41 done
consumer 48 done
consumer 15 done
consumer 6 done
consumer 45 done
consumer 38 done
consumer 21 done
consumer 14 done
consumer 2 done
consumer 9 done


consumer 5 done


In [13]:
import os
from tqdm import tqdm

mapping = {}
for i in tqdm(range(len(all_texts))):
    filename = f'generate-translate-jawi/{i}.json'
    if not os.path.exists(filename):
        continue
    
    with open(filename) as fopen:
        t = json.load(fopen)
        
    mapping[all_texts[i]] = t

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 2897651/2897651 [03:28<00:00, 13888.08it/s]


In [25]:
import json
from glob import glob
from tqdm import tqdm

roles = {
    'user': '<manusia>',
    'assistant': '<bot>'
}

files = glob('/home/husein/ssd3/soalan-augmentation/*conversation*.jsonl')
files.extend(glob('/home/husein/ssd3/soalan-augmentation/*multiturn*.jsonl'))
files = [f for f in files if 'rag' not in f]
alls = []
for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            inputs = []
            try:
                for no, r in enumerate(l):
                    if no < (len(l) - 1):
                        if l[no + 1]['role'] == 'user':
                            next_text = l[no + 1].get('content_ms') or ''
                        else:
                            next_text = l[no + 1].get('content_ms') or ''
                    else:
                        next_text = ''

                    if r['role'] == 'user':
                        q = r.get('content_ms', '') or ''
                        current_text = q       
                    else:
                        current_text = r.get('content_ms') or ''

                    if l[no - 1]['role'] == 'user':
                        previous_text = l[no - 1].get('content_ms')
                    else:
                        previous_text = l[no - 1].get('content_ms') or ''

                    # bad pairs
                    if r['role'] == 'user' and (len(current_text) < 2 or len(next_text) < 2):
                        # print('a', l, current_text, next_text, '\n')
                        continue
                    if r['role'] == 'assistant' and (len(current_text) < 2 or len(previous_text) < 2):
                        continue

                    # bad pairs
                    if r['role'] == 'user' and current_text[:20].lower() == next_text[:20].lower():
                        # print(no, r, current_text[:20].lower(), next_text[:20].lower())
                        continue
                    if r['role'] == 'assistant' and current_text[:20].lower() == previous_text[:20].lower():
                        # print(no, r, current_text[:20].lower(), previous_text[:20].lower())
                        continue

                    # remove alignments    
                    if r['role'] == 'user' and (any([b in current_text.lower() for b in break_at]) or any([b in next_text.lower() for b in break_at])):
                        # print(current_text, next_text)
                        break
                    if r['role'] == 'assistant' and (any([b in current_text.lower() for b in break_at]) or any([b in previous_text.lower() for b in break_at])):
                        # print(current_text, next_text)
                        break

                    role = roles[r['role']]

                    s = current_text

                    inputs.append((s, r))

                if len(inputs) % 2 != 0:
                    inputs = inputs[:-1]

                if not len(inputs):
                    continue

                if len(inputs) < 2:
                    continue

                outputs = []
                for i in range(0, len(inputs), 2):

                    try:
                        content = inputs[i][1]['content']
                        content_ms = inputs[i][1]['content_ms']

                        if len(set(content_ms.split())) < (len(set(content.split())) / 2):
                            continue
                    except:
                        pass

                    try:
                        content = inputs[i + 1][1]['content']
                        content_ms = inputs[i + 1][1]['content_ms']

                        if len(set(content_ms.split())) < (len(set(content.split())) / 2):
                            continue
                    except:
                        pass
                    
                    l = mapping.get(inputs[i][0])
                    r = mapping.get(inputs[i + 1][0])
                    
                    if l is None or r is None:
                        break

                    outputs.extend([
                        {'role': 'user', 'content_ms': inputs[i][0], 'content_jawi': l},
                        {'role': 'assistant', 'content_ms': inputs[i + 1][0], 'content_jawi': r}
                    ])

                if not len(outputs):
                    continue

                alls.append(outputs)
            except:
                pass

48163it [00:08, 5421.09it/s]
60384it [00:13, 4493.38it/s]
57798it [00:11, 5094.76it/s]
135770it [01:45, 1284.01it/s]
103242it [01:01, 1679.54it/s]


In [26]:
rags = [
    '/home/husein/ssd3/soalan-augmentation/rag-multiturn-chaotic-part2.jsonl',
]

for f in rags:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            context = l['context']
            l = l['chat']
            inputs = []
            try:
                for no, r in enumerate(l):
                    if no < (len(l) - 1):
                        if l[no + 1]['role'] == 'user':
                            next_text = l[no + 1].get('content_ms') or ''
                        else:
                            next_text = l[no + 1].get('content_ms') or ''
                    else:
                        next_text = ''

                    if r['role'] == 'user':
                        q = r.get('content_ms', '') or ''
                        current_text = q
                    else:
                        current_text = r.get('content_ms') or ''

                    if l[no - 1]['role'] == 'user':
                        previous_text = l[no - 1].get('content_ms') or ''
                    else:
                        previous_text = l[no - 1].get('content_ms') or ''

                    # bad pairs
                    if r['role'] == 'user' and (len(current_text) < 2 or len(next_text) < 2):
                        # print('a', l, current_text, next_text, '\n')
                        continue
                    if r['role'] == 'assistant' and (len(current_text) < 2 or len(previous_text) < 2):
                        continue

                    # bad pairs
                    if r['role'] == 'user' and current_text[:20].lower() == next_text[:20].lower():
                        # print(no, r, current_text[:20].lower(), next_text[:20].lower())
                        continue
                    if r['role'] == 'assistant' and current_text[:20].lower() == previous_text[:20].lower():
                        # print(no, r, current_text[:20].lower(), previous_text[:20].lower())
                        continue

                    # remove alignments    
                    if r['role'] == 'user' and (any([b in current_text.lower() for b in break_at]) or any([b in next_text.lower() for b in break_at])):
                        # print(current_text, next_text)
                        break
                    if r['role'] == 'assistant' and (any([b in current_text.lower() for b in break_at]) or any([b in previous_text.lower() for b in break_at])):
                        # print(current_text, next_text)
                        break

                    role = roles[r['role']]

                    if no == 0:
                        current_text = f"{context}\n{current_text}"

                    s = current_text

                    inputs.append((s, r))

                if len(inputs) % 2 != 0:
                    inputs = inputs[:-1]

                if not len(inputs):
                    continue

                if len(inputs) < 3:
                    continue

                outputs = []
                for i in range(0, len(inputs), 2):

                    try:
                        content = inputs[i][1]['content']
                        content_ms = inputs[i][1]['content_ms']

                        if len(set(content_ms.split())) < (len(set(content.split())) / 2):
                            continue
                    except:
                        pass

                    try:
                        content = inputs[i + 1][1]['content']
                        content_ms = inputs[i + 1][1]['content_ms']

                        if len(set(content_ms.split())) < (len(set(content.split())) / 2):
                            continue
                    except:
                        pass

                    outputs.extend([
                        inputs[i][0],
                        inputs[i + 1][0]
                    ])

                if not len(outputs):
                    continue

                data = '\n'.join(outputs).strip()
                if not len(data):
                    continue

                if context not in data:
                    continue

                outputs[0] = outputs[0].split(context)[1]
                
                new_outputs = [{'role': 'context', 'content': context}]
                for i in range(len(outputs)):
                    
                    if outputs[i] not in mapping:
                        break
                        
                    if i % 2 == 0:
                        role = 'user'
                    else:
                        role = 'assistant'
                    
                    new_outputs.append({
                        'role': role, 'content_ms': outputs[i], 'content_jawi': mapping[outputs[i]] 
                    })
                
                if len(new_outputs) % 2 == 0:
                    new_outputs = new_outputs[:-1]

                alls.append(new_outputs)
            except Exception as e:
                print(e)
                pass

169403it [01:11, 2371.78it/s]


In [27]:
len(alls)

553571

In [33]:
with open('synthetic-jawi-conversation.jsonl', 'w') as fopen:
    for l in tqdm(alls):
        l = json.dumps(l)
        if '+' in l:
            continue
        fopen.write(f'{l}\n')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 553571/553571 [00:24<00:00, 22535.35it/s]


In [34]:
!ls -lh synthetic-jawi-conversation.jsonl

-rw-rw-r-- 1 husein husein 13G Jun  12 10:42 synthetic-jawi-conversation.jsonl


In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='synthetic-jawi-conversation.jsonl',
    path_in_repo='synthetic-jawi-conversation.jsonl',
    repo_id='mesolitica/synthetic-jawi-conversation',
    repo_type='dataset',
)

synthetic-jawi-conversation.jsonl:   0%|          | 0.00/13.2G [00:00<?, ?B/s]